In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define paths to the dataset
train_path = "train"
test_path = "test1"

# Initialize lists to hold the images and labels
train_images = []
train_labels = []

# Load the training images and extract labels from filenames
for filename in os.listdir(train_path):
    img = cv2.imread(os.path.join(train_path, filename), cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = cv2.resize(img, (64, 64))
        train_images.append(img)
        if 'cat' in filename.lower():
            train_labels.append(0)
        elif 'dog' in filename.lower():
            train_labels.append(1)
print(len(train_labels))
# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Normalize the pixel values
train_images = train_images / 255.0

# Reshape images to be 2D for SVM input
n_samples, height, width = train_images.shape
train_images = train_images.reshape((n_samples, height * width))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Load the test images
test_images = []
test_filenames = []

for filename in os.listdir(test_path):
    img = cv2.imread(os.path.join(test_path, filename), cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = cv2.resize(img, (64, 64))
        test_images.append(img)
        test_filenames.append(filename)

test_images = np.array(test_images)
test_images = test_images / 255.0
test_images = test_images.reshape((test_images.shape[0], height * width))

In [ ]:
# Create and train the SVM model
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)

In [3]:
# Make predictions on the validation data
y_val_pred = svm.predict(X_val)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy: {:.2f}%".format(accuracy * 100))

Validation Accuracy: 67.24%


In [4]:
# Make predictions on the test data
test_predictions = svm.predict(test_images)

# Output the predictions
for filename, prediction in zip(test_filenames, test_predictions):
    label = 'cat' if prediction == 0 else 'dog'
    print(f"File: {filename} - Predicted: {label}")

File: 823.jpg - Predicted: dog
File: 11981.jpg - Predicted: cat
File: 11759.jpg - Predicted: cat
File: 7078.jpg - Predicted: cat
File: 12250.jpg - Predicted: cat
File: 12244.jpg - Predicted: cat
File: 9041.jpg - Predicted: cat
File: 11995.jpg - Predicted: cat
File: 9727.jpg - Predicted: cat
File: 837.jpg - Predicted: dog
File: 11765.jpg - Predicted: cat
File: 11771.jpg - Predicted: cat
File: 12278.jpg - Predicted: dog
File: 12293.jpg - Predicted: cat
File: 11942.jpg - Predicted: cat
File: 11956.jpg - Predicted: dog
File: 12287.jpg - Predicted: dog
File: 5690.jpg - Predicted: cat
File: 162.jpg - Predicted: cat
File: 6399.jpg - Predicted: dog
File: 5684.jpg - Predicted: cat
File: 12052.jpg - Predicted: dog
File: 2464.jpg - Predicted: cat
File: 1179.jpg - Predicted: dog
File: 11229.jpg - Predicted: cat
File: 10651.jpg - Predicted: cat
File: 9243.jpg - Predicted: cat
File: 12046.jpg - Predicted: cat
File: 11567.jpg - Predicted: cat
File: 3020.jpg - Predicted: dog
File: 8613.jpg - Predicted

In [5]:
import pandas as pd

# Sort the filenames numerically
sorted_filenames = sorted(test_filenames, key=lambda x: int(os.path.splitext(x)[0]))

# Make predictions on the sorted test data
sorted_test_images = [test_images[test_filenames.index(filename)] for filename in sorted_filenames]
sorted_test_images = np.array(sorted_test_images)

test_predictions = svm.predict(sorted_test_images)

# Prepare data for the CSV file
output_data = {
    "Filename": sorted_filenames,
    "Label": test_predictions
}

# Create a DataFrame
output_df = pd.DataFrame(output_data)

# Export to CSV
output_df.to_csv("test_predictions.csv", index=False)

print("Predictions have been exported to test_predictions.csv")

Predictions have been exported to test_predictions.csv
